In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras.datasets import mnist

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam

from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping

from keras import backend as K

img_rows, img_cols = 28, 28

(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.1, # Randomly zoom image 
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=False,  # randomly flip images
            vertical_flip=False)  # randomly flip images

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

#defining these prior to model to increase readability and debugging
batch_size = 64
epochs = 10

accuracies = []

for i in range(5):
    print(f"Training Run {i+1}")
    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last',
                         input_shape=(28,28,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid' ))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same', activation='relu', data_format='channels_last'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), padding='valid', strides=2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    #Optimizer
    optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999 )

    #Compiling the model
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

    #for our case LearningRateScheduler will work great
    reduce_lr = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

    #left out early_stopping parameter as it gets better accuracy
    history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, 
                                  validation_data=(x_test, y_test), verbose=1, 
                                  steps_per_epoch=x_train.shape[0] // batch_size,
                                  callbacks=[reduce_lr])

    # Evaluate the model
    _, accuracy = model.evaluate(x_test, y_test, verbose=0)
    accuracies.append(accuracy)
    print("Accuracy: %.2f%%" % (accuracy * 100))

print("Mean Accuracy:", np.mean(accuracies))



Training Run 1




Epoch 1/10


C:\Users\Admin\AppData\Local\Temp\ipykernel_8560\3050234883.py:100: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,


937/937 [==============================] - 85s 89ms/step - loss: 0.3061 - accuracy: 0.9106 - val_loss: 0.0379 - val_accuracy: 0.9888 - lr: 0.0010
Epoch 2/10
937/937 [==============================] - 83s 89ms/step - loss: 0.1090 - accuracy: 0.9670 - val_loss: 0.0472 - val_accuracy: 0.9846 - lr: 9.0000e-04
Epoch 3/10
937/937 [==============================] - 86s 92ms/step - loss: 0.0859 - accuracy: 0.9743 - val_loss: 0.0275 - val_accuracy: 0.9909 - lr: 8.1000e-04
Epoch 4/10
937/937 [==============================] - 84s 90ms/step - loss: 0.0722 - accuracy: 0.9782 - val_loss: 0.0241 - val_accuracy: 0.9921 - lr: 7.2900e-04
Epoch 5/10
937/937 [==============================] - 83s 89ms/step - loss: 0.0609 - accuracy: 0.9817 - val_loss: 0.0212 - val_accuracy: 0.9926 - lr: 6.5610e-04
Epoch 6/10
937/937 [==============================] - 87s 93ms/step - loss: 0.0517 - accuracy: 0.9838 - val_loss: 0.0235 - val_accuracy: 0.9934 - lr: 5.9049e-04
Epoch 7/10
937/937 [=============================

Accuracy: 99.49%
Training Run 2
Epoch 1/10
937/937 [==============================] - 85s 89ms/step - loss: 0.2950 - accuracy: 0.9139 - val_loss: 0.0456 - val_accuracy: 0.9852 - lr: 0.0010
Epoch 2/10
937/937 [==============================] - 87s 93ms/step - loss: 0.1077 - accuracy: 0.9674 - val_loss: 0.0440 - val_accuracy: 0.9856 - lr: 9.0000e-04
Epoch 3/10
937/937 [==============================] - 91s 97ms/step - loss: 0.0854 - accuracy: 0.9745 - val_loss: 0.0338 - val_accuracy: 0.9888 - lr: 8.1000e-04
Epoch 4/10
937/937 [==============================] - 88s 94ms/step - loss: 0.0688 - accuracy: 0.9789 - val_loss: 0.0298 - val_accuracy: 0.9895 - lr: 7.2900e-04
Epoch 5/10
937/937 [==============================] - 90s 96ms/step - loss: 0.0631 - accuracy: 0.9816 - val_loss: 0.0264 - val_accuracy: 0.9918 - lr: 6.5610e-04
Epoch 6/10
937/937 [==============================] - 92s 98ms/step - loss: 0.0531 - accuracy: 0.9843 - val_loss: 0.0353 - val_accuracy: 0.9900 - lr: 5.9049e-04
Epoch 

Epoch 1/10
937/937 [==============================] - 92s 95ms/step - loss: 0.2979 - accuracy: 0.9123 - val_loss: 0.0409 - val_accuracy: 0.9865 - lr: 0.0010
Epoch 2/10
937/937 [==============================] - 92s 99ms/step - loss: 0.1063 - accuracy: 0.9681 - val_loss: 0.0244 - val_accuracy: 0.9925 - lr: 9.0000e-04
Epoch 3/10
937/937 [==============================] - 99s 106ms/step - loss: 0.0816 - accuracy: 0.9758 - val_loss: 0.0733 - val_accuracy: 0.9786 - lr: 8.1000e-04
Epoch 4/10
937/937 [==============================] - 91s 97ms/step - loss: 0.0671 - accuracy: 0.9795 - val_loss: 0.0241 - val_accuracy: 0.9917 - lr: 7.2900e-04
Epoch 5/10
937/937 [==============================] - 90s 96ms/step - loss: 0.0623 - accuracy: 0.9808 - val_loss: 0.0318 - val_accuracy: 0.9913 - lr: 6.5610e-04
Epoch 6/10
937/937 [==============================] - 91s 97ms/step - loss: 0.0540 - accuracy: 0.9835 - val_loss: 0.0260 - val_accuracy: 0.9919 - lr: 5.9049e-04
Epoch 7/10
937/937 [=================

Epoch 1/10
937/937 [==============================] - 97s 99ms/step - loss: 0.2957 - accuracy: 0.9137 - val_loss: 0.0431 - val_accuracy: 0.9863 - lr: 0.0010
Epoch 2/10
937/937 [==============================] - 83s 89ms/step - loss: 0.1108 - accuracy: 0.9671 - val_loss: 0.0273 - val_accuracy: 0.9907 - lr: 9.0000e-04
Epoch 3/10
937/937 [==============================] - 83s 88ms/step - loss: 0.0858 - accuracy: 0.9741 - val_loss: 0.0463 - val_accuracy: 0.9849 - lr: 8.1000e-04
Epoch 4/10
937/937 [==============================] - 85s 91ms/step - loss: 0.0699 - accuracy: 0.9789 - val_loss: 0.0338 - val_accuracy: 0.9896 - lr: 7.2900e-04
Epoch 5/10
937/937 [==============================] - 88s 94ms/step - loss: 0.0610 - accuracy: 0.9817 - val_loss: 0.0178 - val_accuracy: 0.9936 - lr: 6.5610e-04
Epoch 6/10
937/937 [==============================] - 86s 92ms/step - loss: 0.0568 - accuracy: 0.9825 - val_loss: 0.0155 - val_accuracy: 0.9955 - lr: 5.9049e-04
Epoch 7/10
937/937 [==================

Accuracy: 99.52%
Training Run 5
Epoch 1/10
937/937 [==============================] - 88s 92ms/step - loss: 0.2975 - accuracy: 0.9135 - val_loss: 0.0673 - val_accuracy: 0.9799 - lr: 0.0010
Epoch 2/10
937/937 [==============================] - 86s 92ms/step - loss: 0.1076 - accuracy: 0.9679 - val_loss: 0.0410 - val_accuracy: 0.9875 - lr: 9.0000e-04
Epoch 3/10
937/937 [==============================] - 84s 90ms/step - loss: 0.0844 - accuracy: 0.9741 - val_loss: 0.0266 - val_accuracy: 0.9920 - lr: 8.1000e-04
Epoch 4/10
937/937 [==============================] - 88s 94ms/step - loss: 0.0690 - accuracy: 0.9792 - val_loss: 0.0342 - val_accuracy: 0.9899 - lr: 7.2900e-04
Epoch 5/10
937/937 [==============================] - 83s 89ms/step - loss: 0.0584 - accuracy: 0.9833 - val_loss: 0.0193 - val_accuracy: 0.9930 - lr: 6.5610e-04
Epoch 6/10
937/937 [==============================] - 84s 89ms/step - loss: 0.0510 - accuracy: 0.9846 - val_loss: 0.0200 - val_accuracy: 0.9940 - lr: 5.9049e-04
Epoch 